In [ ]:
loaded_modules=0

In [ ]:
import os
import sys
import numpy as np
import swyft
import pickle
import matplotlib.pyplot as plt
import torch
import importlib
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor
torch.set_float32_matmul_precision('medium')
device_notebook = "cuda" if torch.cuda.is_available() else "cpu"
import wandb
import copy
from torch.multiprocessing import Pool
torch.multiprocessing.set_start_method('spawn',force=True)
torch.set_num_threads(28)
import itertools
import subprocess

In [ ]:
main_dir = "ALPs_with_SWYFT"
results_dir_local=os.getcwd()
thesis_figs = os.getcwd().split(main_dir)[0]+"/"+main_dir+"/thesis_figures/"

In [ ]:
config_trial = None
physics_trial = None
truncation_trial = None
store_trial = None
net_trial = None
grid_point=0

#######################################################################################################################

in_archive = True if len(results_dir_local.split("/archive/")) != 1 else False
if config_trial == None:
    config_path = results_dir_local+"/config_variables.pickle"
elif config_trial>-1:
    config_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(physics_trial)+"/config_variables.pickle"
else:
    config_path = results_dir_local.split("/archive/")[0]+"/config_variables.pickle"
if physics_trial == None:
    physics_path = results_dir_local+"/physics_variables.pickle"
elif physics_trial>-1:
    physics_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(physics_trial)+"/physics_variables.pickle"
else:
    physics_path = results_dir_local.split("/archive/")[0]+"/physics_variables.pickle"
if truncation_trial == None:
    truncation_path = results_dir_local+"/truncation_record.pickle"
elif truncation_trial>-1:
    truncation_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(truncation_trial)+"/truncation_record.pickle"
else:
    truncation_path = results_dir_local.split("/archive/")[0]+"/truncation_record.pickle"
if store_trial == None:
    store_path = results_dir_local+"/sim_output/store" if not in_archive else results_dir_local+"/store"
elif store_trial>-1:
    store_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(store_trial)+"/store"
else:
    store_path = results_dir_local.split("/archive/")[0]+"/sim_output/store"
if net_trial == None:
    net_path = results_dir_local+"/train_output/net" if not in_archive else results_dir_local+"/net"
elif net_trial>-1:
    net_path = results_dir_local.split("/archive/")[0]+"/archive/trial__"+str(net_trial)+"/net"
else:
    net_path = results_dir_local.split("/archive/")[0]+"/train_output/net"

if loaded_modules:
    importlib.reload(network)
    importlib.reload(ALP_quick_sim)
    importlib.reload(alp_swyft_simulator)
    importlib.reload(param_func)
    importlib.reload(DRP_test)
    
sys.path.append(results_dir_local)
import ALP_quick_sim
ALP_sim = ALP_quick_sim.ALP_sim
import alp_swyft_simulator 
ALP_SWYFT_Simulator = alp_swyft_simulator.ALP_SWYFT_Simulator
sys.path.append(physics_path)
import param_function as param_func
param_function = param_func.param_function
sys.path.append(net_path)
import network 
NetworkCorner = network.NetworkCorner
Network1D = network.Network1D
Network2D = network.Network2D
loaded_modules=1   
import DRP_test
get_drp_coverage = DRP_test.get_drp_coverage
draw_DRP_samples = DRP_test.draw_DRP_samples
plot_DRP_coverage = DRP_test.plot_DRP_coverage

In [ ]:
with open(config_path, 'rb') as file:
    config_objects = pickle.load(file)
for key in config_objects.keys():
    locals()[key] = config_objects[key]
with open(physics_path, 'rb') as file:
    config_objects = pickle.load(file)
for key in config_objects.keys():
    locals()[key] = config_objects[key]
with open(truncation_path, 'rb') as file:
    truncation_objects = pickle.load(file)
for key in truncation_objects.keys():
    locals()[key] = truncation_objects[key]
filename_true_obs = "true_obs.pickle"
with open(results_dir_local+'/' +filename_true_obs, 'rb') as file:
    true_obs_dict = pickle.load(file)
for key in true_obs_dict.keys():
    locals()[key] = true_obs_dict[key]
results_dir = results_dir_local

In [ ]:
sim = ALP_SWYFT_Simulator(A, bounds, prior_funcs)
sim_trunc = ALP_SWYFT_Simulator(A, bounds_rounds[which_grid_point][which_truncation], prior_funcs)

In [ ]:
bounds

In [ ]:
prior_funcs

In [ ]:
colors = [
    'k',
    'm',
    'r',
    '#FFA500',
    'y',
    'g',
    'c',
    'b',
]

standard_observations_file = results_dir_local.split('/analysis_results/')[0] + "/standard_observations.pickle"

import_observations = [] #[standard_observations_file]


extra_truths = []

#######################################################################################################################
observations = [true_obs]
truths = [true_obs['params']]
truths.extend(extra_truths)
for tru in extra_truths:
    sim.A.configure_model(ALP_seed=0)
    obs_exp=sim.generate_exp(tru)
    obs_data = sim.generate_data(obs_exp,tru)
    obs_power = sim.generate_power(obs_data)
    observations.append(swyft.Sample(power=obs_power, data=obs_data, exp=obs_exp, params=tru))
sim.A.configure_model(ALP_seed=None)

for truth_file  in import_observations:
    with open(truth_file, 'rb') as file:
        observation_objects = pickle.load(file)
    standard_observations = observation_objects['observations']
    for obs in standard_observations:
        observations.append(obs)
        truths.append(obs['params'])

# print(truths)

In [ ]:
store = swyft.ZarrStore(store_path + "/" + store_name+"_round_"+str(which_truncation)+"_gridpoint_"+str(grid_point))
samples = store.get_sample_store()

In [ ]:
len(store)

In [ ]:
plt.figure(figsize=(8,2))
for i in range(len(bounds)):
    plt.subplot(1,len(samples['params'][i]),i+1)
    plt.hist(samples['params'][:,i])

In [ ]:
len(samples['params'][0])

In [ ]:
A.configure_plot(xmin=None, xmax=None, ymin=None,ymax=None,legend=0, logx=1, logy=0)
for i in range(min(len(samples),100)):
    A.configure_model(params=samples[i]['params'])
    A.import_counts(obs=samples[i]['data'],exp=samples[i]['exp'])
    new_fig = 1 if i==0 else 0
    A.compute_case(new_fig=new_fig,new_counts=0,color=None,color_obs=None,linestyle="-",
                   label_exp=0,label_obs=False, errorbands=False, errors=False, plot_exp=0, transparency=0.7)
    
for i,obs in enumerate(observations):
    A.configure_model(params=obs['params'])
    A.import_counts(obs=obs['data'],exp=obs['exp'])
    shade = i/(len(observations)+1)
    A.compute_case(new_fig=0,new_counts=0,color='k',color_obs=colors[i],linestyle_obs="-",label_exp=0,label_obs=0,
                   errorbands=0, errors=0, plot_exp=0)
    A.compute_case(new_fig=0,new_counts=0,color=colors[i],color_obs=colors[i],linestyle=":",label_exp=0,label_obs=0,
                        errorbands=0, errors=0, plot_obs=0) 

In [ ]:
prior_samples = sim.sample(400_000, targets=['params'])
prior_samples_trunc = sim_trunc.sample(400_000, targets=['params'])

In [ ]:
count = 0
for combo in itertools.product(*hyperparams.values()):
    if count == grid_point:
        hyperparams_point = {}
        print("Model number " + str(count))
        for i, key in enumerate(hyperparams.keys()):
            hyperparams_point[key]=combo[i]
    count +=1
hyperparams_point

In [ ]:
if restricted_posterior == 1 or len(POI_indices)==1:
    new_net = Network1D(nbins=A.nbins, marginals=POI_indices, param_names=A.param_names)
elif restricted_posterior == 2:
    new_net = Network2D(nbins=A.nbins, marginals=POI_indices, param_names=A.param_names)
else:
    new_net = NetworkCorner(nbins=A.nbins, marginals=POI_indices, param_names=A.param_names, **hyperparams_point)

new_net.load_state_dict(torch.load(net_path+'/trained_network'+"_round_"+str(which_truncation)+"_gridpoint_"+str(grid_point)+'.pt'))

In [ ]:
lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = swyft.SwyftTrainer(
    accelerator = device_notebook, precision = 64, #logger=wandb_logger, # min_epochs =25,
    callbacks = [lr_monitor], log_every_n_steps=10,
)

In [ ]:
logratios = list(np.zeros(len(truths)))
for j in range(len(truths)):
    plt.figure(figsize=(10,2))
    logratios[j] = trainer.infer(
                            new_net,
                            observations[j],
                            prior_samples_trunc
                            )

In [ ]:
colors_truncations = [
    'k',
    'r',
    # 'y',
    'g',
    'b',
]


for j in range(len(truths)):
    if j == 0:
        for lri in range(len(logratios_rounds[grid_point])-1):
            if lri == 0:
                fig=swyft.plot_posterior(logratios_rounds[grid_point][lri][0], 
                             np.array(A.param_names)[POI_indices], 
                             truth={A.param_names[i]:truths[j][i] for i in POI_indices}, 
                             color_truth='k',color = colors_truncations[lri+1],
                             contours = False,bins=400,
                            )
            else:
                swyft.plot_posterior(logratios_rounds[grid_point][lri][0], 
                             np.array(A.param_names)[POI_indices], 
                             truth={A.param_names[i]:truths[j][i] for i in POI_indices}, 
                             color_truth='k',color = colors_truncations[lri+1],
                             fig = fig,contours = False,bins=400,
                            )
                
            for ai, ax in enumerate(fig.axes):
                ax.axvline(bounds_rounds[grid_point][lri+1][ai][0], color=colors_truncations[lri+1],linestyle=':')
                ax.axvline(bounds_rounds[grid_point][lri+1][ai][1], color=colors_truncations[lri+1],linestyle=':')
                if lri > 0: 
                    ax.axvspan(bounds_rounds[grid_point][lri][ai][0],bounds_rounds[grid_point][lri+1][ai][0], alpha = 0.25, color=colors_truncations[lri+1])
                    ax.axvspan(bounds_rounds[grid_point][lri][ai][1],bounds_rounds[grid_point][lri+1][ai][1], alpha = 0.25, color=colors_truncations[lri+1])

            print()
        
        swyft.plot_posterior(logratios[j], 
                         np.array(A.param_names)[POI_indices], 
                         truth={A.param_names[i]:truths[j][i] for i in POI_indices}, 
                         color_truth=colors[j],contours=False,fig=fig,bins=400,
                        )
        for ai, ax in enumerate(fig.axes):
            ax.axvspan(bounds_rounds[grid_point][-2][ai][0],bounds_rounds[grid_point][-2][ai][1], alpha = 0.25, color=colors_truncations[0])
            ax.set_xlim(bounds_rounds[grid_point][0][ai])
            
        fig2 = copy.deepcopy(fig)
        for ai, ax in enumerate(fig2.axes):
            ax.set_xlim(bounds_rounds[grid_point][1][ai])

In [ ]:
if not (restricted_posterior == 1 or len(POI_indices))==1: 
    for j in range(len(truths)):
        swyft.plot_corner(logratios[j], 
                          np.array(A.param_names)[POI_indices], 
                          truth={A.param_names[i]:truths[j][i] for i in POI_indices},
                          color_truth=colors[j],
                          smooth=3,
                          figsize=(11,11)
                         )
        

In [ ]:
def obs_comparison(obs_vec, sample_vec,widths):
  
    return np.sum(((obs_vec-sample_vec)/widths)**2,axis=-1)

def find_obs_matches(target_obs, n=None, bounds=bounds, samples=samples, component_to_compare="exp",component_indices=[]):
    if not n: n = 5
    if len(component_indices)==0: component_indices = np.arange(samples[component_to_compare].shape[1])

    widths = np.max(samples[component_to_compare][:,component_indices],axis=0)-np.min(samples[component_to_compare][:,component_indices],axis=0)
    
    matches = np.argsort(obs_comparison(np.array([target_obs[component_to_compare]])[:,component_indices], samples[component_to_compare][:,component_indices], widths))[:n]
        
    return matches


def plot_obs_matches(target_obs, n=None, color='r', bounds=bounds, samples=samples, component_to_compare="data",component_indices=[]):
    if not n: n = 5
    matches = list(np.zeros(n))
    matches = find_obs_matches(target_obs, n=n, bounds=bounds, samples=samples,component_to_compare=component_to_compare, component_indices=component_indices)
    fig_hist = plt.figure(figsize=(12,12))
    for i in range(len(bounds)):
        fig_hist.add_subplot(6,3,i+1)
        # plt.subplot(1,len(bounds),i+1)
        plt.hist(samples['params'][matches,i], range=bounds[i],bins=50)
        plt.title(A.param_names[i])
        plt.axvline(target_obs['params'][i], color='r')
    A.configure_plot(xmin = None, xmax = None, ymin=None, ymax=None, logx=1, legend = 1)
    A.compute_case(new_fig=1,null=1,plot_obs=0,color="k",linestyle="--", label_exp=1)
    for j, match in enumerate(matches):
        A.configure_model(params=samples["params"][match])
        A.import_counts(exp=samples["exp"][match],obs=samples["data"][match])
        A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color=color,label_exp=0, errorbands=False, transparency=0.5)    
    
    return matches

In [ ]:
%%time
n_vis = 10
bounds_vis = copy.deepcopy(bounds)
target_obs = observations[0]
n_samples = max(min(10_000_000,n_sim_train),n_vis)
print(n_samples)

matches = plot_obs_matches(target_obs, n_vis, bounds=bounds_vis, samples=samples[:n_samples],
                           component_to_compare='exp', component_indices=[])
A.import_counts(exp=observations[0]["exp"])
A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color='k',label_exp=0, errorbands=False, transparency=0.5)

In [ ]:
def find_POI_matches(POI_indices, target_values, ns=None, bounds=bounds, samples=samples):
    if not ns: ns = np.ones(len(POI_indices))*5
    matches = list(np.zeros(len(POI_indices)))#,dtype=int))
    max_is = list(np.zeros(len(POI_indices)))#,dtype=int))
    for i in range(len(POI_indices)): 
        matches[i] = np.full(ns[i], None)
        max_is[i] = 0
    for samp_i in np.arange(0,len(samples)):
        for j in range(len(POI_indices)):
            current_max = abs(samples["params"][matches[j][max_is[j]]][POI_indices[j]]-target_values[j]) if not np.any(matches[j]==None) else np.inf
            if abs(samples['params'][samp_i][POI_indices[j]]-target_values[j]) <= current_max:
                within_bounds = 1
                if not ( np.any(samples["params"][samp_i,:] < np.array(bounds)[:,0]) or np.any(samples["params"][samp_i,:] > np.array(bounds)[:,1]) ):
                    nones = np.where(matches[j]==None)[0]
                    if len(nones)==0:
                        matches[j][max_is[j]] = int(samp_i)
                        max_is[j] = int(np.argmax(abs(samples["params"][matches[j]][:,POI_indices[j]]-target_values[j]))) 
                    else:
                        max_is[j] = int(nones[0])
                        matches[j][max_is[j]] = int(samp_i)
                        if max_is[j]==ns[j]-1: matches[j] = [int(x) for x in matches[j]]

    for i in range(len(POI_indices)): 
        matches[i] = np.array(matches[i])[np.array(matches[i])!=None]
        if len(matches[i])==0:
            raise ValueError("There were no matches to these bounds!")
        
    return matches

def plot_POI_matches(POI_indices, target_values, ns=None, colors=['r','g','c','b'], bounds=bounds, samples=samples):
    if not ns: ns = np.ones(len(POI_indices))*5
    matches = list(np.zeros(len(POI_indices)))
    matches = find_POI_matches(POI_indices, target_values, ns=ns, bounds=bounds, samples=samples)
    plt.figure(figsize=(8,2))
    for i in range(len(POI_indices)):
        plt.subplot(1,len(POI_indices),i+1)
        plt.hist(samples['params'][matches[i],POI_indices[i]], range=bounds[POI_indices[i]],bins=50)
        plt.title(A.param_names[POI_indices[0]] + " = " + str(target_values[i]))
    A.configure_plot(xmin = None, xmax = None, ymin=None, ymax=None, logx=1, legend = 1)
    A.compute_case(new_fig=1,null=1,plot_obs=0,color="k",linestyle="--", label_exp=1)
    for i, _ in enumerate(matches):
        for j, match in enumerate(matches[i]):
            A.configure_model(params=samples["params"][match])
            A.import_counts(exp=samples["exp"][match])
            A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color=colors[i],label_exp=0, errorbands=False, transparency=0.5)    
    
    return matches

In [ ]:
%%time
n_vis = 50
bounds_vis = copy.deepcopy(bounds)
target_values = copy.deepcopy(bounds)
target_values[0][1] = np.log10(10)
target_values[1][1] = np.log10(0.8)
n_samples = min(100_000,n_sim_train)
for index in POI_indices:
    matches = plot_POI_matches([index, index],[target_values[index][0], target_values[index][1]], [n_vis,n_vis], bounds=bounds_vis, samples=samples[:n_samples])
    A.import_counts(exp=observations[0]["exp"])
    A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color='k',label_exp=0, errorbands=False, transparency=0.5)

In [ ]:
# %%time
# n_vis = 20
# margin = 0.05
# bounds_vis = copy.deepcopy(bounds)
# for b, bound in enumerate(bounds_vis):
#     bounds_vis[b][0] = truths[0][b]-margin*(bounds[b][1]-bounds[b][0])
#     bounds_vis[b][1] = truths[0][b]+margin*(bounds[b][1]-bounds[b][0])
# for index in [0,1]:
#     bounds_vis[index]=copy.deepcopy(bounds[index])
# print(truths)
# print(bounds_vis)
# target_values = copy.deepcopy(bounds)
# target_values[0][1] = np.log10(10)
# target_values[1][1] = np.log10(0.8)
# n_samples = min(100_000,n_sim_train)
# for index in POI_indices:
#     matches = plot_POI_matches([index, index],[target_values[index][0], target_values[index][1]], [n_vis,n_vis], bounds=bounds_vis, samples=samples[:n_samples])
#     A.import_counts(exp=observations[0]["exp"])
#     A.compute_case(new_fig=0,new_counts=0,plot_obs=0,color='k',label_exp=0, errorbands=False, transparency=0.5)

In [ ]:
coverage_samples = trainer.test_coverage(new_net, samples[-min(n_sim_coverage,1_000):], prior_samples_trunc[:min(len(prior_samples_trunc),10_000)])

In [ ]:
columns = 3
columns = min(len(POI_indices),columns)
rows = int(np.ceil(len(POI_indices)/columns))
#fix, axes = plt.subplots(rows, columns, figsize = (12, 4*rows))
fix = plt.figure(figsize = (12, 4*rows))
for i in range(len(POI_indices)):
    j = int(i/columns)
    k = i%columns
    fix.add_subplot(rows, columns, i+1)
    swyft.plot_pp(coverage_samples, 
                  np.array(A.param_names)[POI_indices][i], 
                  ax = fix.axes[i])
plt.tight_layout()

In [ ]:
columns = 3
columns = min(len(POI_indices),columns)
rows = int(np.ceil(len(POI_indices)/columns))
fix = plt.figure(figsize = (12, 4*rows))
for i in range(len(POI_indices)):
    j = int(i/columns)
    k = i%columns
    fix.add_subplot(rows, columns, i+1)
    swyft.plot_zz(coverage_samples, 
                  np.array(A.param_names)[POI_indices][i], 
                  ax = fix.axes[i])
plt.tight_layout()

In [ ]:
if not (restricted_posterior == 1 or len(POI_indices))==1: 
    marginals_2d = []
    for i, el in enumerate(POI_indices[:-1]):
        for j in np.arange(i+1,len(POI_indices)):
            marginals_2d.append( (el,POI_indices[j]) )
    marginals_2d = tuple(marginals_2d)

In [ ]:
if not (restricted_posterior == 1 or len(POI_indices))==1: 
    columns = 3
    columns = min(len(marginals_2d),columns)
    rows = int(np.ceil(len(marginals_2d)/columns))
    fix = plt.figure(figsize = (12, 4*rows))
    for i in range(len(marginals_2d)):
        j = int(i/columns)
        k = i%columns
        fix.add_subplot(rows, columns, i+1)
        swyft.plot_pp(coverage_samples, 
                      [[np.array(param_names)[POI_indices][j] for j in marginal] for marginal in marginals_2d ][i], 
                      ax = fix.axes[i])
    plt.tight_layout()

In [ ]:
if not (restricted_posterior == 1 or len(POI_indices))==1: 
    columns = 3
    columns = min(len(marginals_2d),columns)
    rows = int(np.ceil(len(marginals_2d)/columns))
    fix = plt.figure(figsize = (12, 4*rows))
    for i in range(len(marginals_2d)):
        j = int(i/columns)
        k = i%columns
        fix.add_subplot(rows, columns, i+1)
        swyft.plot_zz(coverage_samples, 
                      [[np.array(param_names)[POI_indices][j] for j in marginal] for marginal in marginals_2d ][i], 
                      ax = fix.axes[i])
    plt.tight_layout()

In [ ]:
%%time

n_draws = DRP_coverage_parameters[0][0]
n_coverage = DRP_coverage_parameters[0][1]
n_sim_coverage_start = DRP_coverage_parameters[0][2]
n_processes = 5

#######################################################################################################################

DRP_draws_file=results_dir+"/train_output/net/DRP_draws_"+str(n_draws)+"_"+str(n_coverage)+"_"+str(n_sim_coverage_start)+"_"+str(grid_point)+".pickle"
if os.path.exists(DRP_draws_file):
    print("Loading draws")
    with open(DRP_draws_file, 'rb') as file:
        draws_objects = pickle.load(file)
    draws1d = draws_objects['draws1d']
    draws2d = draws_objects['draws2d']

else: 
    if __name__=='__main__':
        print("Generating new draws")
        iterable = list(np.zeros(n_processes))
        extra_step = 1 if not n_coverage%n_processes==0 else 0
        for itb in range(len(iterable)):
            range_start = -n_sim_coverage+n_sim_coverage_start + itb*(int(n_coverage/n_processes)+extra_step)
            range_stop = range_start + int(n_coverage/n_processes)+extra_step
            range_stop = range_stop if range_stop < -n_sim_coverage+n_sim_coverage_start+n_coverage else -n_sim_coverage+n_sim_coverage_start+n_coverage
            iterable[itb] = (
                                samples[range_start:range_stop],   
                                   n_draws,
                                   net_path+'/trained_network_'+str(grid_point)+'.pt',
                                   device_notebook,
                                   bounds,
                                   A.nbins,
                                   POI_indices,
                                   A.param_names,
                                   8,
                                   1000,
                                   n_processes,
                hyperparams_point,
                               )
        
        with Pool(n_processes) as pool:
            try:
                res = pool.map(draw_DRP_samples,iterable,chunksize = 1,)
            except Exception as err:
                pool.terminate()
                print(err)
    
    draws1d = { key : np.concatenate([ res[i][0][key] for i in range(len(res))],axis=1) for key in res[0][0].keys() }
    draws2d = { key : np.concatenate([ res[i][1][key] for i in range(len(res))],axis=1) for key in res[0][1].keys() }
    
    with open(DRP_draws_file.split('.pickle')[0],'wb') as file:
            pickle.dump(dict(
                draws1d=draws1d,
                draws2d=draws2d,
            ), file)

In [ ]:
def references1D_1(samples):
    return np.ones((len(samples),1))*0.2

def references1D_2(samples):
    return np.ones((len(samples),1))*0.5

def references1D_3(samples):
    return np.ones((len(samples),1))*0.8


def references1D_4(samples):
    mins = np.min(samples['data'], axis=0)
    maxes = np.max(samples['data'], axis=0)
    refs = np.sum((samples['data'] - mins)/(maxes-mins),axis=-1)
    max_ref = np.max(refs)
    min_ref = np.min(refs)
    return ((refs-min_ref)/(max_ref-min_ref))[:,np.newaxis]

def references1D_5(samples):
    mins = np.min(samples['data'], axis=0)
    maxes = np.max(samples['data'], axis=0)
    refs = np.sum(np.log(samples['data'] - mins+0.0001)/(maxes-mins),axis=-1)
    max_ref = np.max(refs)
    min_ref = np.min(refs)
    return ((refs-min_ref)/(max_ref-min_ref))[:,np.newaxis]

def references1D_6(samples):
    refs = np.exp(1./(references1D_4(samples)+1))
    max_ref = np.max(refs)
    min_ref = np.min(refs)
    return ((refs-min_ref)/(max_ref-min_ref))
    
###########################################################################################

def references2D_1(samples):
    return np.ones((len(samples),2))*0.2

def references2D_2(samples):
    return np.ones((len(samples),2))*0.5

def references2D_3(samples):
    return np.ones((len(samples),2))*0.8


def references2D_4(samples):
    half1 = int(len(samples['data'][0])/2)
    half2 = len(samples['data'][0])-half1
    mins1 = np.min(samples['data'][:,:half1], axis=0)
    maxes1 = np.max(samples['data'][:,:half1], axis=0)
    mins2 = np.min(samples['data'][:,half1:], axis=0)
    maxes2 = np.max(samples['data'][:,half1:], axis=0) 
    refs1 = np.sum((samples['data'][:,:half1] - mins1)/half1*(maxes1-mins1),axis=-1)
    refs2 = np.sum((samples['data'][:,half1:] - mins2)/half2*(maxes2-mins2),axis=-1)
    max_ref1 = np.max(refs1)
    min_ref1 = np.min(refs1)
    max_ref2 = np.max(refs2)
    min_ref2 = np.min(refs2)
    
    return np.array( [((refs1-min_ref1)/(max_ref1-min_ref1)),((refs2-min_ref2)/(max_ref2-min_ref2))] ).transpose()

def references2D_5(samples):
    half1 = int(len(samples['data'][0])/2)
    half2 = len(samples['data'][0])-half1
    mins1 = np.min(samples['data'][:,:half1], axis=0)
    maxes1 = np.max(samples['data'][:,:half1], axis=0)
    mins2 = np.min(samples['data'][:,half1:], axis=0)
    maxes2 = np.max(samples['data'][:,half1:], axis=0) 
    refs1 = np.sum(np.log(samples['data'][:,:half1]-mins1+0.0001)/(maxes1-mins1),axis=-1)
    refs2 = np.sum(np.log(samples['data'][:,half1:]-mins2+0.0001)/(maxes2-mins2),axis=-1)
    max_ref1 = np.max(refs1)
    min_ref1 = np.min(refs1)
    max_ref2 = np.max(refs2)
    min_ref2 = np.min(refs2)
    
    return np.array( [((refs1-min_ref1)/(max_ref1-min_ref1)),((refs2-min_ref2)/(max_ref2-min_ref2))] ).transpose()

def references2D_6(samples):
    refs = np.exp(1./(references2D_4(samples)+1))
    max_ref = np.max(refs)
    min_ref = np.min(refs)
    return ((refs-min_ref)/(max_ref-min_ref))

In [ ]:
ref_colors = [
    'b',
    'c',
    'g',
    'y',
    'r',
    'm',
]

In [ ]:
start_ref = -n_sim_coverage+n_sim_coverage_start
stop_ref = -n_sim_coverage+n_sim_coverage_start+n_coverage if -n_sim_coverage+n_sim_coverage_start+n_coverage<0 else None

In [ ]:
references_1d = [
    references1D_1(samples[start_ref:stop_ref]), 
    references1D_2(samples[start_ref:stop_ref]), 
    references1D_3(samples[start_ref:stop_ref]), 
    references1D_4(samples[start_ref:stop_ref]), 
    references1D_5(samples[start_ref:stop_ref]), 
    references1D_6(samples[start_ref:stop_ref]), 
]

references_2d = [
    references2D_1(samples[start_ref:stop_ref]), 
    references2D_2(samples[start_ref:stop_ref]), 
    references2D_3(samples[start_ref:stop_ref]), 
    references2D_4(samples[start_ref:stop_ref]), 
    references2D_5(samples[start_ref:stop_ref]), 
    references2D_6(samples[start_ref:stop_ref]), 
]

In [ ]:
columns = 3
fig_refs = plt.figure(figsize=(12,6))
for i in range(len(references_1d)):
        fig_refs.add_subplot(int(np.ceil(len(references_1d)/columns)),columns,i+1)
        plt.hist(references_1d[i],color=ref_colors[i], range=[0,1])

In [ ]:
columns = 3
fig_refs = plt.figure(figsize=(12,6))
for i in range(len(references_2d)):
        fig_refs.add_subplot(int(np.ceil(len(references_2d)/columns)),columns,i+1)
        plt.scatter(references_2d[i][:,0],references_2d[i][:,1],color=ref_colors[i])
        plt.xlim(0,1)
        plt.ylim(0,1)

In [ ]:
plt.ioff()
fig_DRP, figs_cov = plot_DRP_coverage(references_1d,references_2d,draws1d,draws2d,
                         [[samples['params'][start_ref:stop_ref,[i]] for i in POI_indices],
                          [samples['params'][start_ref:stop_ref,list(marg)] for marg in marginals_2d]],
                         [A.param_names, ['['+A.param_names[marg[0]]+','+A.param_names[marg[1]]+']' for marg in marginals_2d]],
                         [[np.array(bounds)[i] for i in POI_indices],[np.array(bounds)[list(marg)] for marg in marginals_2d]],
                         ref_colors=ref_colors,
                        )
plt.ion()

In [ ]:
for ax in fig_DRP.axes[1:]:
    ax.set_visible(True)
fig_DRP

In [ ]:
for ax in fig_DRP.axes[1:]:
    ax.set_visible(False)
fig_DRP

In [ ]:
figs_cov[0][-1]['m']

In [ ]:
figs_cov[1][-1]['[m,g]']

In [ ]:
bounds=np.array(bounds)
draws1d_noinfo = { key : np.random.rand(n_draws,n_coverage,1)*(bounds[i,1]-bounds[i,0]) + bounds[i,0] for i, key in enumerate(draws1d.keys()) }
draws2d_noinfo = { key : np.random.rand(n_draws,n_coverage,2)*(bounds[marginals_2d[i],1]-bounds[marginals_2d[i],0]) + bounds[marginals_2d[i],0] for i,key in enumerate(draws2d.keys()) }

In [ ]:
plt.ioff()
fig_DRP_noinfo, figs_cov_noinfo = plot_DRP_coverage(references_1d,references_2d,draws1d_noinfo,draws2d_noinfo,
                         [[samples['params'][start_ref:stop_ref,[i]] for i in POI_indices],
                          [samples['params'][start_ref:stop_ref,list(marg)] for marg in marginals_2d]],
                         [A.param_names, ['['+A.param_names[marg[0]]+','+A.param_names[marg[1]]+']' for marg in marginals_2d]],
                         [[np.array(bounds)[i] for i in POI_indices],[np.array(bounds)[list(marg)] for marg in marginals_2d]],
                         ref_colors=ref_colors,
                        )
plt.ion()

In [ ]:
for ax in fig_DRP_noinfo.axes[1:]:
    ax.set_visible(True)
fig_DRP_noinfo

In [ ]:
for ax in fig_DRP_noinfo.axes[1:]:
    ax.set_visible(False)
fig_DRP_noinfo

In [ ]:
figs_cov_noinfo[0][-1]['m']

In [ ]:
figs_cov_noinfo[1][-1]['[m,g]']